In [1]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, size=5000, dim=40, random_offset=0):
        super(MyDataset, self).__init__()
        self.size = size
        self.dim = dim
        self.random_offset = random_offset

    def __getitem__(self, index):
        if index >= len(self):
            raise IndexError("{} index out of range".format(self.__class__.__name__))

        rng_state = torch.get_rng_state()
        torch.manual_seed(index + self.random_offset)

        while True:
            img = torch.zeros(self.dim, self.dim)
            dx = torch.randint(-10,10,(1,),dtype=torch.float)
            dy = torch.randint(-10,10,(1,),dtype=torch.float)
            c = torch.randint(-20,20,(1,), dtype=torch.float)

            params = torch.cat((dy/dx, c))
            xy = torch.randint(0,img.shape[1], (20, 2), dtype=torch.float)
            xy[:,1] = xy[:,0] * params[0] + params[1]

            xy.round_()
            xy = xy[ xy[:,1] > 0 ]
            xy = xy[ xy[:,1] < self.dim ]
            xy = xy[ xy[:,0] < self.dim ]

            for i in range(xy.shape[0]):
                x, y = xy[i][0], self.dim - xy[i][1]
                img[int(y), int(x)]=1
            if img.sum() > 2:
                break
                
        torch.set_rng_state(rng_state)
        return img.unsqueeze(0), params

    def __len__(self):
        return self.size

train_data = MyDataset()
val_data = MyDataset(size=500, random_offset=33333)
test_data = MyDataset(size=500, random_offset=99999)

# automatically reload external modules if they change
%load_ext autoreload
%autoreload 2

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchbearer
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchbearer import Trial

# create data loaders
trainloader = DataLoader(train_data, batch_size=128, shuffle=True)
testloader = DataLoader(test_data, batch_size=128, shuffle=True)

In [2]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 48, (3, 3), padding=1)
        self.fc1 = nn.Linear(76800, 128)
        self.fc2 = nn.Linear(128, 2)
            
    def forward(self, x): # Input: 128x1x40x40 
        out = self.conv1(x)
        # conv1 outputs 128x48x40x40
        out = F.relu(out)
        out = out.view(out.shape[1], -1)
        #print(out.shape) # torch.Size([128, 76800])
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out
    
class SimpleCNN2(nn.Module):
    def __init__(self):
        super(SimpleCNN2, self).__init__()
        self.conv1 = nn.Conv2d(1, 48, (3, 3), padding=1)
        self.conv2 = nn.Conv2d(1, 48, (3, 3), padding=1)
        self.pool = nn.AdaptiveAvgPool2d((1,76800))
        self.fc1 = nn.Linear(76800, 128)
        self.fc2 = nn.Linear(128, 2)
            
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = self.conv2(x)
        out = F.relu(out)
        #print(out.shape) # torch.Size([128, 48, 40, 40])
        out = self.pool(out)
        #out = out.view(out.shape[0], -1)
        #print(out.shape) # torch.Size([128, 48, 76800, 1])
        out = self.fc1(out[2])
        out = F.relu(out)
        out = self.fc2(out)
        return out

class BetterCNN(nn.Module):
    def __init__(self):
        super(BetterCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 48, (3, 3), padding=1, stride=1)
        self.conv2 = nn.Conv2d(48, 48, (3, 3), padding=1, stride=1)
        self.pool = nn.AdaptiveAvgPool2d((1, 20 * 20))
        self.fc1 = nn.Linear(48 * 1 * (20 * 20), 128)
        self.fc2 = nn.Linear(128, 2)
            
    def forward(self, x):
        # input = 128 x 1 x 40 x 40 
        out = self.conv1(x)
        out = F.relu(out)
        # conv1 output = 128 x 48 x 40 x 40 
        out = self.conv2(out)
        out = F.relu(out)
        # conv2 output = 128 x 48 x 40 x 40 
        out = self.pool(out)
        # print(out.shape)
        out = out.view(out.shape[0], -1)
        # print(out.shape)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        # print(out.shape,"\n\n")
        return out

In [4]:


num_epochs = 100
device = "cuda:0" if torch.cuda.is_available() else "cpu"
#model = SimpleCNN2().to(device)
model = BetterCNN().to(device)

#loss_function = nn.MSELoss(reduction='sum')
optimiser = optim.Adam(model.parameters())

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    train_loss = 0.0
    model.train()
    for data in trainloader:

        inputs, labels = data[0].to(device), data[1].to(device)
        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()

        train_loss += loss.cpu().detach().numpy()
    train_loss = train_loss / (len(trainloader) * 128) 
    train_losses.append(train_loss)

    val_loss = 0.0 
    model.eval()
    for data in testloader:

        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        val_loss += loss.cpu().detach().numpy()
    val_loss = val_loss / (len(testloader) * 128)
    val_losses.append(val_loss)

    torch.cuda.empty_cache()
    
    print("Epoch %d, train_loss %4.2f, val_loss %4.2f" % (epoch, train_loss, val_loss))
print("Finished Training")



RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target' in call to _thnn_nll_loss_forward

In [ ]:
print(train_data[0])

In [4]:
# build the model
model = SimpleCNN()

# define the loss function and the optimiser
loss_function = nn.CrossEntropyLoss()
#loss_function = nn.MSELoss()
optimiser = optim.Adam(model.parameters())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader, test_generator=testloader)
trial.run(epochs=100)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)


RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target' in call to _thnn_nll_loss_forward